In [1]:
import requests, json
from pprint import pprint

In [2]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 11051108}


In [3]:
# get player data from 'elements' field
players = r['elements']

# show data for first player
pprint(players[0])

{'assists': 0,
 'bonus': 0,
 'bps': 5,
 'chance_of_playing_next_round': 100,
 'chance_of_playing_this_round': 100,
 'clean_sheets': 0,
 'clean_sheets_per_90': 0.0,
 'code': 58822,
 'corners_and_indirect_freekicks_order': None,
 'corners_and_indirect_freekicks_text': '',
 'cost_change_event': 0,
 'cost_change_event_fall': 0,
 'cost_change_start': -3,
 'cost_change_start_fall': 3,
 'creativity': '1.6',
 'creativity_rank': 406,
 'creativity_rank_type': 146,
 'direct_freekicks_order': None,
 'direct_freekicks_text': '',
 'dreamteam_count': 0,
 'element_type': 2,
 'ep_next': '0.0',
 'ep_this': '0.5',
 'event_points': 0,
 'expected_assists': '0.00979',
 'expected_assists_per_90': 0.03147,
 'expected_goal_involvements': '0.00979',
 'expected_goal_involvements_per_90': 0.03147,
 'expected_goals': '0.00000',
 'expected_goals_conceded': '0.04350',
 'expected_goals_conceded_per_90': 0.13982,
 'expected_goals_per_90': 0.0,
 'first_name': 'Cédric',
 'form': '0.0',
 'form_rank': 451,
 'form_rank_typ

In [4]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [5]:
# create players dataframe
players = pd.json_normalize(r['elements'])

# show some information about first five players
players[['id', 'web_name', 'team', 'element_type']].head()

,id,web_name,team,element_type
0,1,Cédric,1,2
1,3,Xhaka,1,3
2,4,Elneny,1,3
3,5,Holding,1,2
4,6,Partey,1,3


In [6]:
# create teams dataframe
teams = pd.json_normalize(r['teams'])

teams.head()

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1220,1270,1240,1250,1200,1270,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,3,None,False,0,1090,1100,1110,1130,1090,1110,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,3,None,False,0,1060,1090,1070,1130,1050,1080,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,3,None,False,0,1100,1130,1100,1110,1130,1160,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,3,None,False,0,1150,1160,1140,1160,1170,1190,131


In [7]:
# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])

positions.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],72
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],236
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],296
3,4,Forwards,FWD,Forward,FWD,3,1,3,False,[],83


In [8]:
# join players to teams
df = pd.merge(
    left=players,
    right=teams,
    left_on='team',
    right_on='id'
)

# show joined result
df[['first_name', 'second_name', 'name']].head()

,first_name,second_name,name
0,Cédric,Alves Soares,Arsenal
1,Granit,Xhaka,Arsenal
2,Mohamed,Elneny,Arsenal
3,Rob,Holding,Arsenal
4,Thomas,Partey,Arsenal


In [9]:
# join player positions
df = df.merge(
    positions,
    left_on='element_type',
    right_on='id'
)

# rename columns
df = df.rename(
    columns={'name':'team_name', 'singular_name':'position_name'}
)

# show result
df[
    ['first_name', 'second_name', 'team_name', 'position_name']
].head()

,first_name,second_name,team_name,position_name
0,Cédric,Alves Soares,Arsenal,Defender
1,Rob,Holding,Arsenal,Defender
2,Kieran,Tierney,Arsenal,Defender
3,Benjamin,White,Arsenal,Defender
4,Takehiro,Tomiyasu,Arsenal,Defender


In [10]:
# get data from 'element-summary/{PID}/' endpoint for PID=4
r = requests.get(base_url + 'element-summary/4/').json()

# show top-level fields for player summary
pprint(r, depth=1)

{'fixtures': [...], 'history': [...], 'history_past': [...]}


In [11]:
# show data for first gameweek
pprint(r['history'][0])

{'assists': 0,
 'bonus': 0,
 'bps': 0,
 'clean_sheets': 0,
 'creativity': '0.0',
 'element': 4,
 'expected_assists': '0.00000',
 'expected_goal_involvements': '0.00000',
 'expected_goals': '0.00000',
 'expected_goals_conceded': '0.00000',
 'fixture': 1,
 'goals_conceded': 0,
 'goals_scored': 0,
 'ict_index': '0.0',
 'influence': '0.0',
 'kickoff_time': '2022-08-05T19:00:00Z',
 'minutes': 0,
 'opponent_team': 7,
 'own_goals': 0,
 'penalties_missed': 0,
 'penalties_saved': 0,
 'red_cards': 0,
 'round': 1,
 'saves': 0,
 'selected': 86132,
 'starts': 0,
 'team_a_score': 2,
 'team_h_score': 0,
 'threat': '0.0',
 'total_points': 0,
 'transfers_balance': 0,
 'transfers_in': 0,
 'transfers_out': 0,
 'value': 45,
 'was_home': False,
 'yellow_cards': 0}


In [12]:
def get_gameweek_history(player_id):
    '''get all gameweek info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history' data from response into dataframe
    df = pd.json_normalize(r['history'])
    
    return df


# show player #4's gameweek history
get_gameweek_history(4)[
    [
        'round',
        'total_points',
        'minutes',
        'goals_scored',
        'assists'
    ]
].head()

,round,total_points,minutes,goals_scored,assists
0,1,0,0,0,0
1,2,0,0,0,0
2,3,0,0,0,0
3,4,2,90,0,0
4,5,0,0,0,0


In [13]:
def get_season_history(player_id):
    '''get all past season info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history_past' data from response into dataframe
    df = pd.json_normalize(r['history_past'])
    
    return df


# show player #1's gameweek history
get_season_history(1)[
    [
        'season_name',
        'total_points',
        'minutes',
        'goals_scored',
        'assists'
    ]
].head(10)

,season_name,total_points,minutes,goals_scored,assists
0,2015/16,86,1965,0,2
1,2016/17,102,2515,0,3
2,2017/18,85,2794,0,3
3,2018/19,52,1493,1,2
4,2019/20,61,1553,1,1
5,2020/21,28,744,0,1
6,2021/22,48,1481,1,1


In [14]:
# select columns of interest from players df
players = players[
    ['id', 'first_name', 'second_name', 'web_name', 'team',
     'element_type']
]

# join team name
players = players.merge(
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop(['team', 'id'], axis=1)

# join player positions
players = players.merge(
    positions[['id', 'singular_name_short']],
    left_on='element_type',
    right_on='id'
).drop(['element_type', 'id'], axis=1)

players.head()

,id_player,first_name,second_name,web_name,name,singular_name_short
0,1,Cédric,Alves Soares,Cédric,Arsenal,DEF
1,5,Rob,Holding,Holding,Arsenal,DEF
2,8,Kieran,Tierney,Tierney,Arsenal,DEF
3,10,Benjamin,White,White,Arsenal,DEF
4,14,Takehiro,Tomiyasu,Tomiyasu,Arsenal,DEF


In [15]:
from tqdm.auto import tqdm
tqdm.pandas()

In [16]:
# get gameweek histories for each player
points = players['id_player'].progress_apply(get_gameweek_history)

# combine results into single dataframe
points = pd.concat(df for df in points)

# join web_name
points = players[['id_player', 'web_name']].merge(
    points,
    left_on='id_player',
    right_on='element'
)

  0%|          | 0/687 [00:00<?, ?it/s]

In [26]:
print(points.shape)
print(points.columns)

(11272, 38)
Index(['id_player', 'web_name', 'element', 'fixture', 'opponent_team',
       'total_points', 'was_home', 'kickoff_time', 'team_h_score',
       'team_a_score', 'round', 'minutes', 'goals_scored', 'assists',
       'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved',
       'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus',
       'bps', 'influence', 'creativity', 'threat', 'ict_index', 'starts',
       'expected_goals', 'expected_assists', 'expected_goal_involvements',
       'expected_goals_conceded', 'value', 'transfers_balance', 'selected',
       'transfers_in', 'transfers_out'],
      dtype='object')


In [70]:
points.head(5)

,id_player,web_name,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out
0,1,Cédric,1,1,7,0,False,2022-08-05T19:00:00Z,0.0,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,45,0,23970,0,0
1,1,Cédric,1,11,10,0,True,2022-08-13T14:00:00Z,4.0,2.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,44,-5169,24193,1361,6530
2,1,Cédric,1,21,3,0,False,2022-08-20T16:30:00Z,0.0,3.0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,44,-4337,20960,879,5216
3,1,Cédric,1,31,9,0,True,2022-08-27T16:30:00Z,2.0,1.0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,43,-2988,18825,577,3565
4,1,Cédric,1,41,2,0,True,2022-08-31T18:30:00Z,2.0,1.0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,43,-1611,17790,405,2016


In [62]:
points['expected_goals'] = pd.to_numeric(points['expected_goals'],errors = 'coerce')
points['expected_assists'] = pd.to_numeric(points['expected_assists'],errors = 'coerce')
points['expected_goal_involvements'] = pd.to_numeric(points['expected_goal_involvements'],errors = 'coerce')
points['expected_goals_conceded'] = pd.to_numeric(points['expected_goals_conceded'],errors = 'coerce')

In [66]:
# get top scoring players
points.groupby(
    ['element', 'web_name']
).agg(
    {'total_points':'sum', 'goals_scored':'sum', 'assists':'sum'}
).reset_index(
).sort_values(
    'total_points', ascending=False
).head(10)

,element,web_name,total_points,goals_scored,assists
317,318,Haaland,144,21,3
426,427,Kane,127,15,4
356,357,Trippier,124,1,5
368,369,Almirón,106,9,2
6,7,Ødegaard,103,7,5
79,80,Toney,100,12,3
282,283,Salah,98,7,5
18,19,Martinelli,98,7,5
300,301,De Bruyne,96,3,10
12,13,Saka,94,6,7


In [67]:
# get top scoring players
points.groupby(
    ['element', 'web_name']
).agg(
    {'expected_goals':'mean', 'expected_assists':'mean', 'expected_goal_involvements':'mean'}
).reset_index(
).sort_values(
    'expected_goal_involvements', ascending=False
).head(10)

,element,web_name,expected_goals,expected_assists,expected_goal_involvements
317,318,Haaland,0.850988,0.127069,0.978057
282,283,Salah,0.519571,0.171902,0.691472
426,427,Kane,0.581967,0.090449,0.672416
300,301,De Bruyne,0.180859,0.467843,0.648702
79,80,Toney,0.533417,0.078093,0.611509
209,210,Mitrović,0.534084,0.046693,0.580777
27,28,Jesus,0.464688,0.101363,0.566051
6,7,Ødegaard,0.282912,0.244846,0.527758
296,297,Darwin,0.431782,0.091351,0.523133
12,13,Saka,0.339694,0.143399,0.483093


In [71]:
haaland = points[points['web_name']=='Haaland']

In [72]:
haaland

,id_player,web_name,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out
9625,318,Haaland,318,10,19,13,False,2022-08-07T15:30:00Z,0.0,2.0,1,77,2,0,1,0,0,0,0,0,0,0,3,48,62.6,4.8,73.0,14.0,1,1.6551,0.08401,1.73911,0.4874,115,0,3398599,0,0
9626,318,Haaland,318,17,3,5,True,2022-08-13T14:00:00Z,4.0,0.0,2,73,0,1,1,0,0,0,0,0,0,0,0,16,18.2,13.3,23.0,5.5,1,0.1492,0.33152,0.48072,0.0964,116,1113937,5226268,1143813,29876
9627,318,Haaland,318,28,15,6,False,2022-08-21T15:30:00Z,3.0,3.0,3,90,1,0,0,3,0,0,0,0,0,0,0,28,43.0,11.9,70.0,12.5,1,0.6852,0.01052,0.69572,1.8093,117,227317,5676608,368864,141547
9628,318,Haaland,318,37,7,17,True,2022-08-27T14:00:00Z,4.0,2.0,4,83,3,0,0,2,0,0,0,0,0,0,3,73,87.2,1.3,105.0,19.4,1,1.4134,0.00703,1.42043,0.1176,117,99625,5967805,231905,132280
9629,318,Haaland,318,49,16,17,True,2022-08-31T18:30:00Z,6.0,0.0,5,68,3,0,1,0,0,0,0,0,0,0,3,81,97.2,0.8,91.0,18.9,1,1.6371,0.04728,1.68438,0.3697,118,624916,6793855,675165,50249
9630,318,Haaland,318,51,2,9,False,2022-09-03T16:30:00Z,1.0,1.0,6,90,1,0,0,1,0,0,0,0,0,0,3,32,41.6,14.8,79.0,13.5,1,0.8554,0.27737,1.13277,0.3436,119,782279,7781487,802502,20223
9631,318,Haaland,318,80,20,6,False,2022-09-17T11:30:00Z,0.0,3.0,8,90,1,0,1,0,0,0,0,0,0,0,0,29,37.6,3.4,22.0,6.3,1,0.1261,0.01475,0.14085,0.4100,120,135042,8407273,142869,7827
9632,318,Haaland,318,88,14,23,True,2022-10-02T13:00:00Z,6.0,3.0,9,90,3,2,0,3,0,0,0,0,0,0,3,102,135.2,45.5,124.0,30.5,1,1.1117,0.75574,1.86744,1.6687,121,94190,8548463,126753,32563
9633,318,Haaland,318,97,17,6,True,2022-10-08T14:00:00Z,4.0,0.0,10,90,1,0,1,0,0,0,0,0,0,0,0,23,33.6,21.0,73.0,12.8,1,0.7088,0.05588,0.76468,0.1561,122,272643,8905419,277921,5278
9634,318,Haaland,318,106,12,2,False,2022-10-16T15:30:00Z,1.0,0.0,11,90,0,0,0,1,0,0,0,0,0,0,0,-2,18.2,13.9,122.0,15.4,1,0.4594,0.09232,0.55172,2.0701,122,40632,8989666,72323,31691
